Load model and embedding data.

In [1]:
import theano
import theano.tensor as T
import numpy as np
import cPickle
import random
from utils import *
from state import *
from attention_model import AttentionModel

theano.config.floatX='float32'

model_name = 'model/attention_emb256_h512_v2_model.npz'

state = prototype_state()
model = AttentionModel(state, test_mode=True)
model.load(model_name)

(word2ind, ind2word) = cPickle.load(open('tmp/dict.pkl'))

print('Data loaded.')

Data loaded.


Now we try to mannually provide an input sentence (if a word is out of vocab, we simply remove it).

In [4]:
test_sents = ['moderately priced swedish food', 'how about asian oriental', 'chinese food', \
              'moderately australian food', 'is there any serving vietnamese']

for (k, test_sent) in enumerate(test_sents):
    words = test_sent.split()
    nat_coded = [1]
    for w in words:
        if w in word2ind:
            nat_coded.append(word2ind[w])
        else:
            print '  out of vocab: %s' % w
    nat_coded.append(0)
    #print 'Coded input:', nat_coded

    m = state['seq_len_in']
    nat_coded_mat = numpy.zeros((m, 2), dtype='int32')
    nat_mask = numpy.zeros((m, 2), dtype='float32')
    sent_len = len(nat_coded)
    nat_coded_mat[:sent_len, 0] = nat_coded
    nat_mask[:sent_len, 0] = 1
    pred_fn = theano.function([model.x_data, model.xmask],
                                    model.ot)
    pred = pred_fn(nat_coded_mat, nat_mask)
    #print 'NAT_coded:', nat_coded_mat
    #print 'NAT_mask:', nat_mask
    #print 'Prediction (coded):', pred

    res_col = pred[:, 0]
    print res_col
    
    for col in res_col:
        print col.argmax()

    res = []
    for ind in res_col:
        res.append(ind)
        if ind == 0:
            break

    print "%d: %s\n %s" % (k, test_sent, ' '.join([ind2word[ind] for ind in res]))

[[  4.25117277e-02   1.52548282e-07   8.61931010e-04 ...,   9.30724607e-04
    9.86388768e-04   9.74392053e-04]
 [  9.99993682e-01   0.00000000e+00   1.22841430e-15 ...,   1.59289734e-15
    3.36927048e-15   2.03691921e-15]
 [  1.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    8.40779079e-45   1.40129846e-45]
 ..., 
 [  1.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  1.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  1.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]]
0
0
0
0
0
0
0
0
0
0


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()